# 同化なしのアンサンブル計算

このノートブックでは，外力に乱数ノイズを与えた50メンバーのアンサンブル計算を観測値への同化なしで行う。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import interpolate
import advdiff

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['mathtext.fontset'] = 'cm'

----

## シミュレーション

In [ ]:
params = {
    'nx': 40,
    'dx': 10.0,
    'dt': 0.5,
    'u': 2.0,
    'nu': 5.0,
    'freq_euler': 15
}

In [ ]:
# 外力項を与える関数の設定
# 時刻を引数として呼び出すと各格子点における外力の値を返す
x = np.arange(params['nx']) * params['dx']

#
frc_x = np.sin(np.pi * x / 60.) * np.where(x < 60., 1.,  0.)
frc_t = lambda t: np.sin(2. * np.pi * t / 120.)

In [ ]:
nt = 1200
freq_out = 15
n_ensemble = 10 # number of ensemble members

model = advdiff.EnsembleModel(n_ensemble, **params)

vda = [model.data]

for it in range(nt):
    _t = it * model.dt
    _ft = frc_t(_t) + np.random.normal(loc=0.0, scale=2.0, size=model.n_ensemble)
    frc = _ft.reshape((-1,1)) @ frc_x.reshape((1,-1))
    model.step_forward(force=frc)
    if model.it % freq_out == 0:
        vda.append(model.data)

vda = np.array(vda)

## 結果のプロット

In [ ]:
# 真値の読み込み
vt = np.loadtxt('./data/state_true.dat')

In [ ]:
it = 18

fig, ax = plt.subplots(figsize=(8,3))
plt.plot(x, vt[it,:], '-', c='C0', lw=2, label='True')
for iens in range(n_ensemble):
    label = 'Simulation' if iens == 0 else '__nolegend__'
    plt.plot(x, vda[it,iens,:], '-', c='C1', lw=0.5, label=label)
plt.xlim(0, 400)
plt.ylim(-40, 40)
plt.xlabel(r'$x$ [km]')
plt.ylabel(r'$C$')
plt.legend(loc='lower right', framealpha=0.7)
plt.grid(ls='--', c='0.5', lw=0.3)
plt.title(r'Free Run ($t={:4.1f}$)'.format(it * 7.5))
plt.tight_layout()